# Engineering AI Model Training

This notebook demonstrates how to fine-tune a language model for engineering knowledge using an NVIDIA RTX 4060 GPU. The model will be trained to answer engineering questions like "How is a car built?" with detailed and accurate responses.

## Prerequisites
- NVIDIA RTX 4060 GPU
- CUDA installed
- Python 3.8+
- Virtual environment `ai-notebooks-env` activated
- Sufficient disk space (at least 10GB for models and data)

## Setup Instructions
Before running this notebook:
1. Create virtual environment: `python -m venv ai-notebooks-env`
2. Activate it: `ai-notebooks-env\Scripts\activate` (Windows)
3. Install Jupyter: `pip install jupyter notebook`
4. Start notebook: `jupyter notebook engineering_ai_model.ipynb`

## 1. Install Required Libraries

First, we need to install the necessary libraries for model training and GPU acceleration.

In [1]:
# Check virtual environment and system prerequisites
import sys
import os
import subprocess

print("🔍 System Check")
print("=" * 50)

# Check Python version
python_version = sys.version_info
print(f"Python version: {python_version.major}.{python_version.minor}.{python_version.micro}")

# Check if Python version is compatible
if python_version < (3, 8):
    print("❌ Python 3.8+ required!")
    print("   Please upgrade your Python version")
elif python_version < (3, 10):
    print("⚠️  Python < 3.10 detected - will use compatible package versions")
    print("   For best compatibility, consider upgrading to Python 3.10+")
else:
    print("✅ Compatible Python version")

# Check if we're in virtual environment
venv_path = os.environ.get('VIRTUAL_ENV')
if venv_path:
    print(f"✅ Virtual environment active: {os.path.basename(venv_path)}")
    if 'ai-notebooks-env' in venv_path:
        print("✅ Correct virtual environment (ai-notebooks-env)")
    else:
        print("⚠️  Different virtual environment detected")
        print(f"   Expected: ai-notebooks-env")
        print(f"   Current: {os.path.basename(venv_path)}")
else:
    print("❌ No virtual environment detected!")
    print("   Please activate ai-notebooks-env:")
    print("   ai-notebooks-env\\Scripts\\activate")

# Check disk space
try:
    import shutil
    free_gb = shutil.disk_usage('.')[2] / (1024**3)
    print(f"Free disk space: {free_gb:.1f} GB")
    if free_gb < 15:
        print("⚠️  Low disk space! Recommend at least 15GB free")
    else:
        print("✅ Sufficient disk space")
except:
    print("Could not check disk space")

# Check CUDA availability (preliminary check)
try:
    result = subprocess.run(['nvidia-smi'], capture_output=True, text=True, shell=True)
    if result.returncode == 0:
        print("✅ NVIDIA GPU detected")
    else:
        print("❌ NVIDIA GPU not detected or drivers not installed")
except:
    print("❌ Could not check GPU status")

print("=" * 50)

🔍 System Check
Python version: 3.9.13
⚠️  Python < 3.10 detected - will use compatible package versions
   For best compatibility, consider upgrading to Python 3.10+
✅ Virtual environment active: ai-notebooks-env
✅ Correct virtual environment (ai-notebooks-env)
Free disk space: 721.7 GB
✅ Sufficient disk space
✅ NVIDIA GPU detected


### Virtual Environment Setup Instructions

If you haven't set up the virtual environment yet, run these commands in PowerShell/CMD:

```bash
# Create virtual environment
python -m venv ai-notebooks-env

# Activate virtual environment (Windows)
ai-notebooks-env\Scripts\activate

# Install Jupyter in the virtual environment
pip install jupyter notebook ipykernel

# Add kernel to Jupyter
python -m ipykernel install --user --name ai-notebooks-env --display-name "AI Notebooks (ai-notebooks-env)"

# Start Jupyter Notebook
jupyter notebook
```

**Important**: Make sure you select the "AI Notebooks (ai-notebooks-env)" kernel when running this notebook!

In [ ]:
# Install required packages in virtual environment with compatibility fixes
import sys

print("📦 Installing required packages...")
print("This may take several minutes for the first run.")
print("=" * 60)

# Check Python version for compatibility
python_version = sys.version_info
is_old_python = python_version < (3, 10)

if is_old_python:
    print(f"⚠️  Python {python_version.major}.{python_version.minor} detected")
    print("   Installing compatible package versions...")

# Upgrade pip first
!python -m pip install --upgrade pip

# Install PyTorch with CUDA support
print("Installing PyTorch with CUDA 12.1...")
!pip install torch==2.1.0 torchvision==0.16.0 torchaudio==2.1.0 --index-url https://download.pytorch.org/whl/cu121

# Install transformers and related libraries with version constraints for older Python
print("Installing Transformers and ML libraries...")
if is_old_python:
    print("   Using Python < 3.10 compatible versions...")
    !pip install "transformers<4.36.0" "datasets<2.15.0" "accelerate<0.25.0"
else:
    !pip install transformers datasets accelerate

# Install additional packages
print("Installing additional libraries...")
!pip install scikit-learn matplotlib seaborn

# Install optional packages (with error handling)
print("Installing optional packages...")
try:
    !pip install wandb --quiet
    print("✅ wandb installed (for experiment tracking)")
except:
    print("⚠️  wandb installation failed (optional)")

# Verify installations
print("\n🔍 Verifying installations...")
try:
    import torch
    print(f"✅ PyTorch {torch.__version__} installed")
    
    import transformers
    print(f"✅ Transformers {transformers.__version__} installed")
    
    import datasets
    print(f"✅ Datasets library installed")
    
    import sklearn
    print(f"✅ Scikit-learn installed")
    
    print("✅ All core packages installed successfully!")
    
    if is_old_python:
        print("\n💡 Note: Using compatibility versions for Python < 3.10")
    
except ImportError as e:
    print(f"❌ Installation error: {e}")
    print("Please check your virtual environment and try again.")

print("=" * 60)

📦 Installing required packages...
This may take several minutes for the first run.
⚠️  Python 3.9 detected
   Installing compatible package versions...
Installing PyTorch with CUDA 12.1...


## 2. Import Dependencies and Check GPU

Let's import the necessary libraries and verify that our NVIDIA RTX 4060 is available for training.

In [ ]:
import torch
import torch.nn as nn
from transformers import (
    AutoTokenizer, AutoModelForCausalLM, 
    TrainingArguments, Trainer, DataCollatorForLanguageModeling
)
from datasets import Dataset, DatasetDict
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import os
import warnings
warnings.filterwarnings('ignore')

# Check GPU availability
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"GPU device: {torch.cuda.get_device_name(0)}")
    print(f"GPU memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
    device = torch.device("cuda")
else:
    print("CUDA not available. Using CPU.")
    device = torch.device("cpu")

## 3. Prepare Training Data

We'll create a dataset of engineering questions and answers to fine-tune our model.

In [ ]:
# Create engineering Q&A dataset
engineering_qa_data = [
    {
        "question": "How is a car built?",
        "answer": "Car manufacturing involves several key stages: 1) Design and Engineering - CAD modeling, prototyping, and testing. 2) Body Manufacturing - stamping steel sheets into body panels, welding them together in the body shop. 3) Paint Shop - cleaning, priming, painting, and clear coating the body. 4) Assembly Line - installing the engine, transmission, electrical systems, interior components, wheels, and final quality checks. The process uses lean manufacturing principles, robotics for precision tasks, and just-in-time inventory management."
    },
    {
        "question": "What are the main components of an internal combustion engine?",
        "answer": "The main components include: 1) Engine Block - houses cylinders and crankshaft. 2) Pistons - move up and down in cylinders to compress fuel-air mixture. 3) Connecting Rods - link pistons to crankshaft. 4) Crankshaft - converts linear piston motion to rotational motion. 5) Valves - control intake of fuel-air mixture and exhaust of combustion gases. 6) Camshaft - operates the valves via cams. 7) Spark Plugs - ignite the compressed fuel-air mixture. 8) Fuel System - delivers fuel to cylinders. 9) Cooling System - prevents overheating."
    },
    {
        "question": "How does a bridge support heavy loads?",
        "answer": "Bridges support loads through structural engineering principles: 1) Load Distribution - spreading weight across multiple support points. 2) Material Strength - using high-strength materials like steel and reinforced concrete. 3) Structural Design - beam bridges use compression and tension, arch bridges use compression, suspension bridges use tension cables. 4) Foundation Systems - deep foundations transfer loads to stable soil or bedrock. 5) Safety Factors - designing for loads much greater than expected maximum. 6) Regular Inspection - monitoring for fatigue, corrosion, and structural integrity."
    },
    {
        "question": "What is the engineering process for designing a building?",
        "answer": "Building design follows a systematic process: 1) Programming - understanding client needs, site analysis, and code requirements. 2) Schematic Design - initial concepts, massing studies, and basic layouts. 3) Design Development - detailed systems integration, structural, mechanical, electrical design. 4) Construction Documents - detailed drawings and specifications for construction. 5) Permitting - obtaining necessary approvals from authorities. 6) Construction Administration - overseeing construction, reviewing submittals, and ensuring quality. Each phase involves coordination between architects, structural, mechanical, electrical, and civil engineers."
    },
    {
        "question": "How do airplane wings generate lift?",
        "answer": "Airplane wings generate lift through aerodynamic principles: 1) Bernoulli's Principle - curved wing shape causes air to move faster over the top surface, creating lower pressure above the wing. 2) Newton's Third Law - wing deflects air downward, creating an equal and opposite upward force (lift). 3) Angle of Attack - tilting the wing increases lift up to the stall angle. 4) Airfoil Shape - specially designed cross-section optimizes lift-to-drag ratio. 5) Wing Design Features - flaps, slats, and winglets enhance performance. The amount of lift depends on air density, wing area, airspeed, and angle of attack."
    },
    {
        "question": "What are the key principles of mechanical engineering?",
        "answer": "Mechanical engineering is built on fundamental principles: 1) Mechanics - statics (forces in equilibrium) and dynamics (motion and forces). 2) Thermodynamics - energy conversion, heat transfer, and fluid mechanics. 3) Materials Science - understanding material properties, strength, fatigue, and failure modes. 4) Design Process - problem identification, concept generation, analysis, optimization, and testing. 5) Manufacturing - processes like machining, casting, welding, and assembly. 6) Control Systems - feedback loops, sensors, actuators for automated systems. 7) Safety and Reliability - factor of safety, failure analysis, and quality control."
    },
    {
        "question": "How does a hydraulic system work?",
        "answer": "Hydraulic systems work on Pascal's principle: 1) Incompressible Fluid - oil or hydraulic fluid transmits pressure equally in all directions. 2) Pressure Multiplication - small force on small piston creates large force on large piston. 3) System Components - reservoir, pump, filters, valves, actuators (cylinders or motors), and connecting lines. 4) Control Valves - directional, pressure, and flow control valves manage system operation. 5) Applications - construction equipment, aircraft controls, automotive brakes, and industrial machinery. Advantages include high power-to-weight ratio, precise control, and ability to hold loads without continuous power input."
    },
    {
        "question": "What is the engineering behind electric motors?",
        "answer": "Electric motors convert electrical energy to mechanical energy: 1) Electromagnetic Induction - current-carrying conductors in magnetic fields experience force. 2) Motor Types - DC motors (brushed/brushless), AC induction motors, synchronous motors, stepper motors. 3) Key Components - stator (stationary part with electromagnets), rotor (rotating part), commutator (in DC motors), bearings. 4) Control Systems - variable frequency drives for AC motors, electronic speed controllers for DC motors. 5) Efficiency Factors - motor design, materials, manufacturing precision, and operating conditions. Applications range from tiny servo motors to large industrial drives."
    }
]

# Convert to DataFrame for easier handling
df = pd.DataFrame(engineering_qa_data)
print(f"Dataset size: {len(df)} question-answer pairs")
print("\nSample questions:")
for i, question in enumerate(df['question'][:3]):
    print(f"{i+1}. {question}")

# Create training format (instruction-following format)
def format_training_data(question, answer):
    return f"Question: {question}\nAnswer: {answer}"

df['text'] = df.apply(lambda row: format_training_data(row['question'], row['answer']), axis=1)

# Split into train and validation
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)
print(f"\nTrain size: {len(train_df)}")
print(f"Validation size: {len(val_df)}")

# Display sample formatted training data
print("\nSample formatted training data:")
print(train_df['text'].iloc[0])

## 4. Load and Configure Base Model

We'll use GPT-2 as our base model and configure it for fine-tuning on engineering knowledge.

In [ ]:
# Load pre-trained model and tokenizer
model_name = "gpt2"  # Using GPT-2 as base model (good for RTX 4060)
print(f"Loading model: {model_name}")

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Add padding token (GPT-2 doesn't have one by default)
tokenizer.pad_token = tokenizer.eos_token
model.resize_token_embeddings(len(tokenizer))

# Move model to GPU if available
model = model.to(device)

print(f"Model loaded successfully!")
print(f"Model parameters: {model.num_parameters():,}")
print(f"Model device: {next(model.parameters()).device}")

# Test tokenization
sample_text = "Question: How is a car built?\nAnswer:"
tokens = tokenizer(sample_text, return_tensors="pt")
print(f"\nSample tokenization:")
print(f"Input text: {sample_text}")
print(f"Token IDs shape: {tokens['input_ids'].shape}")
print(f"Tokens: {tokens['input_ids'][0][:10]}...")  # Show first 10 tokens

In [ ]:
# Prepare dataset for training
def tokenize_function(examples):
    # Tokenize the text
    tokenized = tokenizer(
        examples['text'], 
        truncation=True, 
        padding=True, 
        max_length=512,  # Reasonable length for RTX 4060
        return_tensors="pt"
    )
    # For causal language modeling, labels are the same as input_ids
    tokenized["labels"] = tokenized["input_ids"].clone()
    return tokenized

# Convert to HuggingFace Dataset format
train_dataset = Dataset.from_pandas(train_df[['text']])
val_dataset = Dataset.from_pandas(val_df[['text']])

# Tokenize datasets
train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)

print(f"Train dataset size: {len(train_dataset)}")
print(f"Validation dataset size: {len(val_dataset)}")
print(f"Sample tokenized data keys: {train_dataset[0].keys()}")

# Create data collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,  # We're doing causal language modeling, not masked language modeling
)

## 5. Set Up Training Configuration

Configure training parameters optimized for NVIDIA RTX 4060 (8GB VRAM).

In [ ]:
# Training arguments optimized for RTX 4060
training_args = TrainingArguments(
    output_dir="./engineering-model-checkpoints",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=2,  # Small batch size for 8GB VRAM
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=4,  # Effective batch size = 2 * 4 = 8
    warmup_steps=10,
    learning_rate=5e-5,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=1,
    evaluation_strategy="steps",
    eval_steps=5,
    save_strategy="steps",
    save_steps=10,
    save_total_limit=3,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    fp16=True,  # Use mixed precision to save memory
    dataloader_pin_memory=False,
    report_to=None,  # Disable wandb for now
    remove_unused_columns=False,
)

print("Training configuration:")
print(f"- Epochs: {training_args.num_train_epochs}")
print(f"- Train batch size: {training_args.per_device_train_batch_size}")
print(f"- Gradient accumulation: {training_args.gradient_accumulation_steps}")
print(f"- Effective batch size: {training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps}")
print(f"- Learning rate: {training_args.learning_rate}")
print(f"- Mixed precision (fp16): {training_args.fp16}")

# Create output directory
os.makedirs(training_args.output_dir, exist_ok=True)

## 6. Fine-tune the Model

Now we'll train the model on our engineering dataset.

In [ ]:
# Initialize trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

print("Starting training...")
print(f"GPU memory before training: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")

# Start training
try:
    train_result = trainer.train()
    print("Training completed successfully!")
    
    # Print training results
    print(f"\nTraining results:")
    print(f"- Final training loss: {train_result.training_loss:.4f}")
    print(f"- Training steps: {train_result.global_step}")
    print(f"- Training time: {train_result.metrics['train_runtime']:.2f} seconds")
    
    # Save the final model
    trainer.save_model("./fine-tuned-engineering-model")
    tokenizer.save_pretrained("./fine-tuned-engineering-model")
    print("\nModel saved to './fine-tuned-engineering-model'")
    
except Exception as e:
    print(f"Training failed with error: {e}")
    print("This might be due to memory constraints. Try reducing batch size or sequence length.")

print(f"GPU memory after training: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")

## 7. Test Model with Engineering Questions

Let's test our trained model with various engineering questions.

In [ ]:
# Function to generate answers
def ask_engineering_question(question, max_length=200):
    """Generate an answer to an engineering question"""
    prompt = f"Question: {question}\nAnswer:"
    
    # Tokenize input
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    
    # Generate response
    with torch.no_grad():
        outputs = model.generate(
            inputs.input_ids,
            max_length=len(inputs.input_ids[0]) + max_length,
            num_return_sequences=1,
            temperature=0.7,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )
    
    # Decode the response
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # Extract just the answer part
    if "Answer:" in response:
        answer = response.split("Answer:", 1)[1].strip()
        return answer
    else:
        return response

# Test questions
test_questions = [
    "How is a car built?",
    "What are the main components of a bridge?",
    "How does a jet engine work?",
    "What is the engineering process for designing a skyscraper?",
    "How do solar panels convert sunlight to electricity?",
    "What are the key principles of structural engineering?"
]

print("Testing the fine-tuned model:\n")
print("="*60)

for i, question in enumerate(test_questions, 1):
    print(f"\n{i}. Question: {question}")
    print("-" * 50)
    
    try:
        answer = ask_engineering_question(question)
        print(f"Answer: {answer}")
    except Exception as e:
        print(f"Error generating answer: {e}")
    
    print("="*60)

## 8. Compare with Base Model

Let's compare our fine-tuned model against the original GPT-2 model to see the improvement.

In [ ]:
# Load original GPT-2 for comparison
print("Loading original GPT-2 for comparison...")
base_model = AutoModelForCausalLM.from_pretrained("gpt2").to(device)
base_tokenizer = AutoTokenizer.from_pretrained("gpt2")
base_tokenizer.pad_token = base_tokenizer.eos_token

def ask_base_model(question, max_length=200):
    """Generate answer using base GPT-2 model"""
    prompt = f"Question: {question}\nAnswer:"
    inputs = base_tokenizer(prompt, return_tensors="pt").to(device)
    
    with torch.no_grad():
        outputs = base_model.generate(
            inputs.input_ids,
            max_length=len(inputs.input_ids[0]) + max_length,
            num_return_sequences=1,
            temperature=0.7,
            do_sample=True,
            pad_token_id=base_tokenizer.eos_token_id,
            eos_token_id=base_tokenizer.eos_token_id,
        )
    
    response = base_tokenizer.decode(outputs[0], skip_special_tokens=True)
    if "Answer:" in response:
        answer = response.split("Answer:", 1)[1].strip()
        return answer
    return response

# Compare models on the main question
comparison_question = "How is a car built?"

print(f"Comparison Question: {comparison_question}")
print("="*80)

print("\n🤖 BASE GPT-2 RESPONSE:")
print("-" * 40)
try:
    base_answer = ask_base_model(comparison_question)
    print(base_answer)
except Exception as e:
    print(f"Error: {e}")

print(f"\n🔧 FINE-TUNED MODEL RESPONSE:")
print("-" * 40)
try:
    finetuned_answer = ask_engineering_question(comparison_question)
    print(finetuned_answer)
except Exception as e:
    print(f"Error: {e}")

print("\n" + "="*80)
print("Notice how the fine-tuned model provides more detailed and")
print("engineering-specific responses compared to the base model!")

## 9. Save and Load Trained Model

Instructions for saving and reloading your trained model for future use.

In [ ]:
# Save the trained model (if not already saved)
model_save_path = "./fine-tuned-engineering-model"

print("Saving trained model...")
try:
    # Save model and tokenizer
    model.save_pretrained(model_save_path)
    tokenizer.save_pretrained(model_save_path)
    print(f"✅ Model saved successfully to: {model_save_path}")
    
    # List saved files
    import os
    saved_files = os.listdir(model_save_path)
    print(f"Saved files: {saved_files}")
    
except Exception as e:
    print(f"❌ Error saving model: {e}")

print("\n" + "="*60)
print("To load this model in the future, use:")
print("="*60)
print("""
from transformers import AutoTokenizer, AutoModelForCausalLM

# Load the fine-tuned model
tokenizer = AutoTokenizer.from_pretrained('./fine-tuned-engineering-model')
model = AutoModelForCausalLM.from_pretrained('./fine-tuned-engineering-model')

# Use the model for inference
def ask_question(question):
    prompt = f"Question: {question}\\nAnswer:"
    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(inputs.input_ids, max_length=200, temperature=0.7)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response.split("Answer:", 1)[1].strip()

# Example usage
answer = ask_question("How is a car built?")
print(answer)
""")

## 10. Interactive Q&A Interface

Create a simple interactive interface to test your model with custom questions.

In [ ]:
# Interactive Q&A function
def interactive_qa():
    """Interactive function to ask engineering questions"""
    print("🔧 Engineering AI Assistant")
    print("=" * 50)
    print("Ask me any engineering question! (type 'quit' to exit)")
    print("Examples:")
    print("- How is a car built?")
    print("- What are the main components of a bridge?")
    print("- How does a jet engine work?")
    print("=" * 50)
    
    while True:
        try:
            question = input("\n❓ Your question: ").strip()
            
            if question.lower() in ['quit', 'exit', 'stop']:
                print("👋 Goodbye!")
                break
                
            if not question:
                print("Please enter a question.")
                continue
            
            print("\n🤔 Thinking...")
            answer = ask_engineering_question(question, max_length=150)
            print(f"\n🔧 Answer: {answer}")
            
        except KeyboardInterrupt:
            print("\n👋 Goodbye!")
            break
        except Exception as e:
            print(f"❌ Error: {e}")

# Uncomment the line below to run the interactive interface
# interactive_qa()

# Alternative: Test with a few predefined questions
print("🔧 Engineering AI Assistant - Demo Mode")
print("=" * 50)

demo_questions = [
    "How does a wind turbine generate electricity?",
    "What materials are used in aircraft construction?",
    "How is concrete made stronger?",
]

for question in demo_questions:
    print(f"\n❓ Question: {question}")
    try:
        answer = ask_engineering_question(question, max_length=120)
        print(f"🔧 Answer: {answer}")
    except Exception as e:
        print(f"❌ Error: {e}")
    print("-" * 50)

## Summary and Next Steps

### What we accomplished:
1. ✅ Set up the environment for training on NVIDIA RTX 4060
2. ✅ Created an engineering-focused question-answer dataset
3. ✅ Fine-tuned GPT-2 on engineering knowledge
4. ✅ Tested the model with various engineering questions
5. ✅ Compared performance against the base model
6. ✅ Saved the trained model for future use

### Model Performance:
The fine-tuned model should now provide more detailed and accurate answers to engineering questions compared to the base GPT-2 model. The training data focused on:
- Manufacturing processes (car building, etc.)
- Structural engineering (bridges, buildings)
- Mechanical systems (engines, motors)
- Engineering principles and processes

### Next Steps to Improve the Model:
1. **Expand Training Data**: Add more diverse engineering Q&A pairs
2. **Domain-Specific Training**: Focus on specific engineering domains
3. **Longer Training**: Increase epochs for better convergence
4. **Larger Model**: Try GPT-2 medium or large (if memory allows)
5. **Evaluation Metrics**: Implement proper evaluation metrics
6. **Real-World Testing**: Test with actual engineering problems

### Memory Optimization Tips for RTX 4060:
- Use gradient checkpointing for larger models
- Implement dynamic batching
- Use DeepSpeed for advanced memory optimization
- Consider model quantization techniques

### Deployment Options:
- Create a web interface using Gradio or Streamlit
- Deploy as an API using FastAPI
- Integrate into existing engineering tools